In [13]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

# Инструкция по выполнению

1 Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" 

(инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

2 Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. 
При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.


3 Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).


4 Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.


5 Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.


In [3]:
# загрузим тексты
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

In [55]:
X_raw = newsgroups.data
y = newsgroups.target
# получим TF-IDF-признаки 
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X_raw)

In [64]:
# выберем лучший параметр
svm = SVC(random_state=241, kernel='linear')
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)

gs = GridSearchCV(svm, grid, scoring='accuracy', cv=cv, n_jobs=-1)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [116]:
# вот эти лучшие параметры
gs.best_params_

{'C': 1.0}

In [128]:
# обучим модель с лучшими параметрами без разделения на folds
svm_best = SVC(kernel='linear', random_state=241, C=gs.best_params_['C'])
svm_best.fit(X, y)

SVC(kernel='linear', random_state=241)

In [167]:
# берем коэффициенты признаков по модулю и сортируем их
coefs = abs(svm_best.coef_.todense().A1)
coefs = np.argsort(coefs)

In [190]:
# получаем по индексам фич с наибольшими весами значение самих фич - слов
df = pd.DataFrame(np.transpose(svm_best.coef_.toarray()), index=np.asarray(vectorizer.get_feature_names()) , columns=['col1'])
df['col1']= np.abs(df2['col1'])
df = df.sort_values(by = 'col1', ascending=False)
# требуется только первые 10 в алфавитном порядке
best10_words = df2.index[0:10].sort_values()
best10_words

Index(['atheism', 'atheists', 'bible', 'god', 'keith', 'moon', 'religion',
       'sci', 'sky', 'space'],
      dtype='object')

In [191]:
# выводим в файл слова с разделителем пробелом
out = ''
for i in range(len(best10_words)-1):
    out += best10_words[i]
    if (i < len(best10_words)-2):
        out += ' '
    
f = open('result_2.txt', 'w')
print(out, file=f, end="")
f.close()
print(out)

atheism atheists bible god keith moon religion sci sky
